#HW 4
##Question 5

In [ ]:
import pandas as pd
import numpy as np
import numpy.matlib
import math 
from numpy import linalg as LA
from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
income = pd.read_csv('/content/drive/My Drive/IS/IS_HW4/income_Q5.csv')

In [ ]:
def normalize(x):
    y = (x - np.mean(x, axis = 0))/np.std(x, axis = 0)
    return y

this function splits data to 80 percent train and 20 percent test data.

In [ ]:
def split_data( data ):
    copy = data.to_numpy()
    size = copy.shape[0]
    data = data.sample(frac = 1, random_state =1).sample(frac = 1, random_state =1).sample(frac = 1, random_state =1)
    cross = int(size/5)
    return data.iloc[ cross: , : ] , data.iloc[ :cross , : ]

In [ ]:
income.head()

,age,workclass,education,educational-num,marital-status,occupation,relationship,race,gender,native-country,income
0,<=38,Private,11th,<=9,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States,<=50K
1,<=38,Private,HS-grad,<=9,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States,<=50K
2,<=38,Local-gov,Assoc-acdm,>9,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States,>50K
3,>38,Private,Some-college,>9,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States,>50K
4,<=38,Special,Some-college,>9,Never-married,?,Own-child,White,Female,United-States,<=50K


This function returns array of each column unique attribute array.

In [ ]:
def get_unique_attr( data ):
    size = data.to_numpy().shape[1]
    all_uniques = []
    for i in range(size):
        all_uniques.append(data.iloc[ : , i ].unique())
    all_uniques = np.asarray(all_uniques)
    return all_uniques

This function returns array of dictioneries which each dictionary's key is a columns unique attribute and it's value is a 2 in 1 array that indicates coincedence of income status in last column.

In [ ]:
def create_dict( uniques ):
    a = []
    for i in range( len( uniques ) ):
        temp = uniques[i]
        d = {}
        for j in range( temp.shape[0] ):
            d[ temp[ j ] ] = np.zeros((2,1))
        a.append(d)
    return np.asarray( a )

This function fills previous dictionary with train data.

In [ ]:
def count_income( data ):
    all_uniques = get_unique_attr( data )
    data = data.to_numpy()
    uniq_dict = create_dict( all_uniques )
    for i in range( data.shape[0] ):
        for j in range( data.shape[1] ):
            if data[ i , -1 ] == all_uniques[ -1 ][ 0 ] :
                uniq_dict[ j ][ data[ i , j ] ][ 0 ] += 1
            elif data[ i , -1 ] == all_uniques[ -1 ][ 1 ] :
                uniq_dict[ j ][ data[ i , j ] ][ 1 ] += 1
    return uniq_dict

Based on counted amounts in previous function, this function calculates probablity.
#Laplace Smoothing
It solves this by giving the last word a small non-zero probability for both classes, so that the posterior probabilities don't suddenly drop to zero. to do this, we add one to numenator and add number of columns ( here is 10) to denumerator. it helps the data that has low frequencty in train data to train that it causes many frauds in last decition.

In [ ]:
def cal_probs( dict_prob , all_uniques ):
    for i in range( len( dict_prob ) ):
        for j in range( all_uniques[ i ].shape[0] ):
            dict_prob[ i ][ all_uniques[ i ][ j ] ][ 0 ] = (dict_prob[ i ][ all_uniques[ i ][ j ] ][ 0 ]+1)/( dict_prob[ -1 ][ all_uniques[ -1 ][ 0 ] ][ 0 ] + len( dict_prob ) )
            dict_prob[ i ][ all_uniques[ i ][ j ] ][ 1 ] = (dict_prob[ i ][ all_uniques[ i ][ j ] ][ 1 ]+1)/( dict_prob[ -1 ][ all_uniques[ -1 ][ 1 ] ][ 1 ] + len( dict_prob ) )
    return dict_prob

This funvtion calculate probablity for each row in given data set and decide that which label should be predicted in each row.

In [ ]:
def predict( data , probs ):
    data = data.to_numpy()
    labels = np.zeros((data.shape[0],1))
    for i in range( data.shape[0] ):
        p_1 = 1
        p_0 = 1
        for j in range( data.shape[1] - 1 ):
            p_1 *= probs[ j ][ data[ i , j ] ][ 1 ]
            p_0 *= probs[ j ][ data[ i , j ] ][ 0 ]
        if p_1 > p_0 :
            labels[ i ] = 1
    return labels

Changes each unique values with numbers assigned to it.

In [ ]:
def change_label( labels ):
    u = labels.unique()
    labels = labels.to_numpy()
    temp = np.zeros((labels.shape[0]))
    for j in range(labels.shape[0]):
        for i in range( u.shape[0] ):
            if labels[j] == u[i]:
                temp[ j ] = i
    return temp

Calculcates accuracy.

In [ ]:
def cal_acc( train , test ):
    uniques = get_unique_attr( train )
    features_dict = count_income( train )
    probs = cal_probs( features_dict , uniques )
    predicted = predict( test , probs )
    labels = change_label( test.iloc[ : , -1 ] )
    return (np.sum( labels == predicted[:,0] )*100)/(predicted[:,0].shape[0]) , print_confusion_mat( predicted , labels )

In [ ]:
def print_confusion_mat( predicted , label ):
    keys = np.unique(predicted)
    confusion = np.zeros(( 2 , 2 ))
    for i in range(label.shape[0]):
        confusion[ int(predicted[ i ])-1 , int(label[ i ])-1 ] += 1
    return pd.DataFrame(confusion)

As shown below for givven data set and then deviding it into test and train and evaluating model, final accuracy is 77%.


In [ ]:
train , test = split_data( income )
acc , mat = cal_acc( train , test )
print( acc )

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


76.89393939393939


Confusion matrix is given below.

In [ ]:
mat

,0,1
0,1929.0,1859.0
1,398.0,5582.0
